In [2]:
!pip install -r requirements.txt
!python -m nltk.downloader stopwords
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw-1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 129.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.9/288.9 kB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.0/467.0 kB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.1/944.1 kB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from nltk.corpus import stopwords
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.stem.wordnet import WordNetLemmatizer
import pickle

ModuleNotFoundError: No module named 'nltk'

In [107]:
df = pd.read_csv("Telegram_Prediction.csv")
df

,Unnamed: 0,Date,Prediction,Content
0,0,2022-07-19 13:21:45+00:00,1,$CLV getting ready 👌🏻
1,0,2022-07-19 13:20:33+00:00,1,"$OGN is on the way to $0,3 👀🚀"
2,0,2022-07-19 13:12:12+00:00,1,$FIL is a buy.
3,0,2022-07-19 13:05:41+00:00,1,Buying $QI here!
4,0,2022-07-19 13:03:15+00:00,2,$CHR +13% so far!
...,...,...,...,...
95,0,2022-06-24 17:10:23+00:00,1,Buying some $TRB on this breakout!
96,0,2022-06-24 16:31:27+00:00,2,$OP +20% from entry!
97,0,2022-06-24 11:35:49+00:00,1,$ICP is going to $7
98,0,2022-06-24 10:56:30+00:00,2,The #Bitcoin drawdown from ATH has reached 73....


In [108]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def text_cleaning(text):
    if(text!=None):
        text=str(text)
        stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
        return str(normalized.split())

df["Cleaned Text"]= df["Content"].apply(text_cleaning)
df.drop(columns="Unnamed: 0",inplace=True)

In [109]:
X = df.loc[:,"Cleaned Text"]
Y = df.loc[:,'Prediction']
Y =Y.apply(lambda x : str(x))

In [110]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.1)
model = make_pipeline(TfidfVectorizer(lowercase=False), MultinomialNB())
model.fit(X_train, Y_train)
predict = model.predict(X_test)
model.score(predict,Y_test)

0.6

In [111]:
pickle.dump(model, open('./Telegram', 'wb'))